<a href="https://colab.research.google.com/github/Ares1510/Learning-TensorFlow/blob/master/Cats_vs_Dogs_3_block_VGG_style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from zipfile import ZipFile
import os
import shutil
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
# Data Preprocessing Steps

#Unzipping Files
train_zip = ZipFile('/content/drive/My Drive/dogs-vs-cats/train.zip')
train_zip.extractall('/content')
test_zip = ZipFile('/content/drive/My Drive/dogs-vs-cats/test1.zip')
test_zip.extractall('/content')

#Creating new directories
os.mkdir('/content/dogs-vs-cats')
os.mkdir('/content/dogs-vs-cats/train')
os.mkdir('/content/dogs-vs-cats/test')
os.mkdir('/content/dogs-vs-cats/train/cats')
os.mkdir('/content/dogs-vs-cats/train/dogs')

SRC = '/content/train'
DEST_CATS = '/content/dogs-vs-cats/train/cats'
DEST_DOGS = '/content/dogs-vs-cats/train/dogs'

#Moving train and test files into new directories
files = (file for file in os.listdir(SRC) if os.path.isfile(os.path.join(SRC, file)))
for file in files:
  if 'cat' in Path(SRC + '/' + file).stem:
    shutil.copy(SRC + '/' + file, DEST_CATS + '/' + file)
  else:
    shutil.copy(SRC + '/' + file, DEST_DOGS + '/' + file)

for file in os.listdir('/content/test1'):
  shutil.copy('/content/test1' + '/' + file, '/content/dogs-vs-cats/test' + '/' + file)

In [0]:
#Setting train and test directories for ImageDataGenerator
TRAIN_DIR = '/content/dogs-vs-cats/train/'
TEST_DIR = '/content/dogs-vs-cats/test'

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_gen = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(300,300), batch_size=64, class_mode='binary', subset='training')
validation_gen = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(300,300), batch_size=64, class_mode='binary', subset='validation')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [0]:
#Creating the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.MaxPooling2D((2,2), ))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(train_gen, steps_per_epoch=150, epochs=4, validation_data=validation_gen, validation_steps=150, verbose=1)

Epoch 1/4
150/150 [==============================] - 384s 3s/step - loss: 6.7666 - accuracy: 0.4951 - val_loss: 0.6927 - val_accuracy: 0.5194
Epoch 2/4
150/150 [==============================] - 384s 3s/step - loss: 0.6927 - accuracy: 0.5097 - val_loss: 0.6920 - val_accuracy: 0.4974
Epoch 3/4
150/150 [==============================] - 381s 3s/step - loss: 0.6873 - accuracy: 0.5386 - val_loss: 0.6840 - val_accuracy: 0.5606
Epoch 4/4
150/150 [==============================] - 371s 2s/step - loss: 0.6839 - accuracy: 0.5599 - val_loss: 0.6781 - val_accuracy: 0.5783
